# Step 15: Prompt Rendering Filters

This notebook demonstrates how to use filters to modify prompts before and after rendering.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments
from semantic_kernel.filters import PromptRenderContext, FilterTypes

In [ ]:
# Define system message
system_message = """
You are a chat bot. Your name is RK Bot.
"""

In [ ]:
# Define prompt rendering filter
async def prompt_rendering_filter(context: PromptRenderContext, next):
    print("Before Rendering...")
    await next(context)
    print("After Rendering...")
    
    # Modify the rendered prompt
    context.rendered_prompt = f"In case you found any city names, just say, I can answer about countries and not city like 'Just give me a country name and not city' {context.rendered_prompt or ''}"

In [ ]:
# Initialize kernel and add filter
kernel = Kernel()
service_id = "chat-gpt"
kernel.add_service(AzureChatCompletion(service_id=service_id))
kernel.add_filter(FilterTypes.PROMPT_RENDERING, prompt_rendering_filter)

settings = kernel.get_prompt_execution_settings_from_service_id(service_id)

In [ ]:
# Create chat function
chat_function = kernel.add_function(
    plugin_name="ChatBot",
    function_name="Chat",
    prompt="{{$chat_history}}{{$user_input}}",
    template_format="semantic-kernel",
    prompt_execution_settings=settings,
)

chat_history = ChatHistory(system_message=system_message)
chat_history.add_user_message("Hi there, who are you?")

In [ ]:
# Interactive chat function
async def chat(user_input: str):
    answer = await kernel.invoke(
        chat_function,
        KernelArguments(
            user_input=user_input,
            chat_history=str(chat_history),
        ),
    )
    chat_history.add_user_message(user_input)
    chat_history.add_assistant_message(str(answer))
    print(f"RK Bot:> {answer}")
    return answer

In [ ]:
# Example: Test the filter
await chat("where am i located?")

In [ ]:
# Interactive chat loop
# Type 'exit' to quit
while True:
    user_input = input("User > ")
    
    if user_input == "exit":
        break
    
    await chat(user_input)